In [1]:
#import the modules
import psycopg2 as pg
import time
from io import StringIO
import pandas as pd
import os
from datetime import datetime
from datetime import date
import numpy as np
from tqdm.notebook import tqdm
import pyspark
import calendar
from pyspark.sql import SparkSession
from pyspark.sql import Row
from datetime import timedelta
from pyspark.sql.functions import regexp_replace
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import date_format
import warnings
warnings.filterwarnings('ignore')

########################################################## CONVERTING 1-min to DIFFERENT TIMEFRAMES - UNDERLYING ######################################

def EOD_underlying(ddf,index):
    print("CONVERTING TO EOD")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], format='mixed',dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='B')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{index}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_EqData\\{index}_EOD.csv",index=False)
    
def fifteen_underlying(ddf,index):
    print("CONVERTING TO 15min")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], format='mixed',dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='15min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{index}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf['Time'] = pd.to_datetime(ddf['Datetime']).dt.time
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','Time','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_EqData\\{index}_15min.csv",index=False)

def five_underlying(ddf,index):
    print("CONVERTING TO 5min")
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], format='mixed',dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df.sort_values(by=['Date'])
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")

    ddf = final_df.groupby(['Date', pd.Grouper(freq='5min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum'})
    ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
    ddf = ddf.reset_index()
    ddf['Ticker'] = f"{index}".upper()+'.EQ-NSE'

    ddf = ddf.sort_values(by=['Datetime'])
    ddf['Time'] = pd.to_datetime(ddf['Datetime']).dt.time
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf.rename(columns={'Adj_Open':'EQ_Open','Adj_High':'EQ_High','Adj_Low':'EQ_Low','Adj_Close':'EQ_Close','Adj_Volume':'EQ_Volume'})
    ddf = ddf[['Ticker','Date','Time','EQ_Open','EQ_High','EQ_Low','EQ_Close','EQ_Volume']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_EqData\\{index}_5min.csv",index=False)

def one_underlying(ddf,index):
    final_df = ddf.copy()
    final_df['Date'] = pd.to_datetime(final_df['Date'], format= 'mixed', dayfirst=True)
    final_df = final_df[(final_df['Time']>=time1) & (final_df['Time']<=time2)]
    final_df = final_df.sort_values(by=['Date','Time'])
    final_df.reset_index(drop=True,inplace=True)
    final_df = final_df[['Ticker','Date','Time','Open','High','Low','Close','Volume']]
    final_df.to_csv(fr"C:\\Users\\admin\\desktop\\{index}_EqData\\{index}_1min.csv",index=False)
    
###################################################### CONVERTING 1-min to DIFFERENT TIMEFRAMES - OPTIONS DATA ##########################################

def EOD(ddf,index,schema,hyphen_index):
    print("CONVERTING TO EOD")
    ddf = ddf.rename(columns={'ticker' : 'Ticker',
                            'date' : 'Date',
                            'time' : 'Time',
                            'open' : 'Open',
                            'high' : 'High', 
                            'low' : 'Low',
                            'close' : 'Close',
                            'volume' : 'Volume', 
                            'Open Int' : 'Open Interest'})
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    symbol = index.upper()
    j='-' + schema[hyphen_index:]
    schema_find = schema[:hyphen_index].upper()
    
    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{index.upper()}'+schema_find, '').str.replace(f'{index.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format = 'mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='B')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol + schema_find + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol + schema_find + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type']

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],dayfirst=True)
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_EOD.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_EOD.csv"), index=False)

def fifteen_min(ddf,index,schema,hyphen_index):
    print("CONVERTING TO 15Min")
    ddf = ddf.rename(columns={'ticker' : 'Ticker',
                            'date' : 'Date',
                            'time' : 'Time',
                            'open' : 'Open',
                            'high' : 'High', 
                            'low' : 'Low',
                            'close' : 'Close',
                            'volume' : 'Volume', 
                            'Open Int' : 'Open Interest'})
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    symbol = index.upper()
    j='-' + schema[hyphen_index:]
    schema_find = schema[:hyphen_index].upper()

    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{index.upper()}'+schema_find, '').str.replace(f'{index.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='15min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol + schema_find + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol + schema_find + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Time'] = pd.to_datetime(df_eod['Date']).dt.time
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],dayfirst=True).dt.date
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_15min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_15min.csv"), index=False)
    
def five_min(ddf,index,schema,hyphen_index):
    print("CONVERTING TO 5Min")
    ddf = ddf.rename(columns={'ticker' : 'Ticker',
                            'date' : 'Date',
                            'time' : 'Time',
                            'open' : 'Open',
                            'high' : 'High', 
                            'low' : 'Low',
                            'close' : 'Close',
                            'volume' : 'Volume', 
                            'Open Int' : 'Open Interest'})
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    
    symbol = index.upper()
    j='-' + schema[hyphen_index:]
    schema_find = schema[:hyphen_index].upper()

    final_df = ddf.copy()
    final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
    final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{index.upper()}'+schema_find, '').str.replace(f'{index.upper()}','').str.replace('CE', '').str.replace('PE', '')
    final_df['Final_strike'] = final_df['Final_strike'].astype(float)
    final_df['Option_Type'] = final_df['Ticker'].str[-2:]
    
    final_df = final_df.rename(columns={'Time' : 'Timestamp',
                                        'Open' : 'Adj_Open',
                                        'High' : 'Adj_High',
                                        'Low' : 'Adj_Low',
                                        'Close' : 'Adj_Close',
                                        'Volume' : 'Adj_Volume',
                                        'Open Interest' : 'Adj_OI',        
                                        'Option_type' : 'Option_Type'})
    final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
    final_df = final_df.sort_values(by=['Date', 'Timestamp'])
    final_df['Date'] = final_df['Date'].astype(str)
    final_df['Timestamp'] = final_df['Timestamp'].astype(str)
    final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

    final_df = final_df.set_index("Datetime")
    final_df['Adj_OI_1'] = final_df['Adj_OI']

    df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='5min')]).agg({"Adj_Open" : "first", 
                                                          "Adj_High" : "max",
                                                          "Adj_Low" : "min",
                                                          "Adj_Close" : "last", 
                                                          'Adj_Volume' : 'sum',
                                                          'Adj_OI' : 'first',
                                                          'Adj_OI_1' : 'last'})
    df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
    df_eod = df_eod.reset_index()
    df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
    df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol + schema_find + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
    df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol + schema_find + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

    df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
    df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
    df_eod['Time'] = pd.to_datetime(df_eod['Date']).dt.time
    df_eod['Date'] = pd.to_datetime(df_eod['Date'],dayfirst=True).dt.date
    
    ## CHECKING IF NULL VALUES
    df_eod['New_OI'] = df_eod['Last_OI']
    df_eod = df_eod.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
    df_eod = df_eod.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
    df_eod = df_eod[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
    df_eod.reset_index(drop=True,inplace=True)
    df_eod.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_5min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_5min.csv"), index=False)

def one_min(ddf,index,schema,hyphen_index):
    ddf = ddf.rename(columns={'ticker' : 'Ticker',
                            'date' : 'Date',
                            'time' : 'Time',
                            'open' : 'Open',
                            'high' : 'High', 
                            'low' : 'Low',
                            'close' : 'Close',
                            'volume' : 'Volume', 
                            'Open Int' : 'Open_Interest'})
    ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
    ddf = ddf.sort_values(by=['Date'])
    ddf = ddf[(ddf['Time']>=time1) & ((ddf['Time']<=time2))]
    ddf = ddf.sort_values(by=['Date','Time'])
    ddf.reset_index(drop=True,inplace=True)
    ddf = ddf[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
    ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_1min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\{index}_{schema}_Data\\{index}_{schema}_Opt_1min.csv"), index=False)
    
########################################################## OPTIONS DATA FUNCTION ###############################################################

def option_data(index,date1,date2,conversion,schema):
    hyphen_index = schema.find("I")

    st=time.time()
    ## CREATING A DIRECTORY OF THE REQUIRED INDEX AND SCHEMA
    if not os.path.exists(rf"C:\\users\\admin\\desktop\\{index}_{schema}_Data\\"):
        os.makedirs(rf"C:\users\admin\desktop\\{index}_{schema}_Data\\")

    date1 = datetime.strptime(date1, "%Y-%m-%d").date()
    date2 = datetime.strptime(date2, "%Y-%m-%d").date()
    year1 = date1.year
    year2 = date2.year
    print("\nGENERATING OPTIONS DATA")
    for i in range(int(year1),int(year2)+1):
        ddate1 = '-01-01'
        ddate2 = '-12-31'
        year_start = str(str(i)+ddate1)
        year_end = str(str(i)+ddate2)
        year_start = datetime.strptime(year_start, "%Y-%m-%d").date()
        year_end = datetime.strptime(year_end, "%Y-%m-%d").date()
        if date1 > year_start :
            year_start = date1
        else:
            year_start = year_start
        if year_end > date2 :
            year_end = date2
        else:
            year_end = year_end

        ddf = pd.DataFrame()
        engine = pg.connect(f"dbname='{index}db' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
        print("Generating data from ", year_start , " to " , year_end)
        ddf = pd.read_sql(f'select * from "{index}{schema}".select_datewise(\'{year_start}\',\'{year_end}\')',con=engine)
        ddf = ddf.sort_values(by=["date",'time'])
        ddf.reset_index(drop=True,inplace=True)
        # with open(f"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv", "w") as file:
        #     cursor.copy_expert(sql, file)
        if conversion == 'E':
            EOD(ddf,index,schema,hyphen_index)
    
        elif conversion == '15':
            fifteen_min(ddf,index,schema,hyphen_index)
    
        elif conversion == '5':
            five_min(ddf,index,schema,hyphen_index)
    
        elif conversion == '1':
            one_min(ddf,index,schema,hyphen_index)

        elif conversion == 'a' or conversion == 'A':
            EOD(ddf,index,schema,hyphen_index)
            fifteen_min(ddf,index,schema,hyphen_index)
            five_min(ddf,index,schema,hyphen_index)
            one_min(ddf,index,schema,hyphen_index)

    et=time.time()
    elapsed_time=et-st;
    print("OPTIONS DATA GENERATED!")
    print("elapsed_time:",elapsed_time)

########################################################## UNDERLYING DATA FUNCTION ###############################################################

def underlying_data(index,date1,date2,conversion):

    if not os.path.exists(rf"C:\\users\\{admin_path}\\desktop\\{index}_EqData\\"):
        os.makedirs(rf"C:\\users\\{admin_path}\\desktop\\{index}_EqData\\")
    
    start_date = datetime.strptime(date1,"%Y-%m-%d")
    end_date = datetime.strptime(date2,"%Y-%m-%d")
    start_date=start_date.date()
    end_date = end_date.date()
    print("\nGENERATING UNDERLYING DATA")
    st = time.time()
    
    engine = pg.connect("dbname='IndexEQ' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
    ddf = pd.read_sql(f'select * from "{index}"."AllData" where "Date" between \'{date1}\' and \'{date2}\'', con=engine)
    ddf['Ticker'] = f'{index.upper()}' + '.EQ-NSE'
    ddf = ddf.sort_values(by=["Date",'Time'])

    if conversion == 'E':
        EOD_underlying(ddf,index)
    elif conversion == '15':
        fifteen_underlying(ddf,index)
    elif conversion == '5':
        five_underlying(ddf,index)
    elif conversion == '1':
        one_underlying(ddf,index)
    elif conversion == 'a' or conversion == 'A':
        EOD_underlying(ddf,index)
        fifteen_underlying(ddf,index)
        five_underlying(ddf,index)
        one_underlying(ddf,index)

    et=time.time()
    elapsed_time=et-st;
    print("UNDERLYING DATA GENERATED!")
    print("elapsed_time:",elapsed_time)
    engine.close()

######################################################## MAIN CODE STARTS FROM HERE ######################################################################
admin_path = 'admin'
time1 = datetime.strptime('09:15:00','%H:%M:%S').time()
time2 = datetime.strptime('15:30:00','%H:%M:%S').time()

data = input("Enter O for Options data, U for Underlying data, B for Both the data, E for Exiting ")

################################################ TAKING INPUTS FOR INDEX, DATE RANGE AND TIMEFRAME #############################################

if data == 'o' or data == 'O' or data == 'b' or data == 'B':
    index = input("Enter the index you want in the format below - \nBankNifty\nNifty\nFinNifty ")
    schema = input("Enter schema (MonthlyI, MonthlyII , WeeklyI , QuarterlyI and so on) - ")
    date1 = input("Enter start date as YYYY-MM-DD ")
    date2 = input("Enter end date as YYYY-MM-DD ")
    conversion = input("Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD, A for All timeframes\n")

elif data == 'u' or data == 'U':
    index = input("Enter the index you want in the format below - \nBankNifty\nNifty\nFinNifty\nIndiaVix ")
    date1 = input("Enter start date as YYYY-MM-DD ")
    date2 = input("Enter end date as YYYY-MM-DD ")
    conversion = input("Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD, A for All timeframes\n")

elif data == 'e' or data == 'E':
    print("Exit!")

else:
    print("Wrong option")

start_time = time.time()
    
if data == 'O' or data == 'o':
    option_data(index,date1,date2,conversion,schema)

elif data == 'U' or data == 'u':
    underlying_data(index,date1,date2,conversion)

elif data == 'B' or data == 'b':
    option_data(index,date1,date2,conversion,schema)
    underlying_data(index,date1,date2,conversion)

end_time = time.time()
print("\nCOMPLETED.")
print("Total time taken ",end_time-start_time)




Enter O for Options data, U for Underlying data, B for Both the data, E for Exiting  B
Enter the index you want in the format below - 
BankNifty
Nifty
FinNifty  BankNifty
Enter schema (MonthlyI, MonthlyII , WeeklyI , QuarterlyI and so on) -  WeeklyI
Enter start date as YYYY-MM-DD  2023-01-02
Enter end date as YYYY-MM-DD  2023-01-02
Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD, A for All timeframes
 1



GENERATING OPTIONS DATA
Generating data from  2023-01-02  to  2023-01-02
OPTIONS DATA GENERATED!
elapsed_time: 49.31162357330322

GENERATING UNDERLYING DATA
UNDERLYING DATA GENERATED!
elapsed_time: 0.6285817623138428

COMPLETED.
Total time taken  49.940205335617065


In [1]:
# import psycopg2 as pg
# import time
# from io import StringIO
# import pandas as pd
# import os
# from datetime import datetime
# from datetime import date
# import numpy as np
# from tqdm.notebook import tqdm
# import pyspark
# import calendar
# from pyspark.sql import SparkSession
# from pyspark.sql import Row
# from datetime import timedelta
# from pyspark.sql.functions import regexp_replace
# from pyspark.sql.functions import array_contains
# from pyspark.sql.functions import date_format
# import warnings
# warnings.filterwarnings('ignore')

# index = 'BankNifty'
# schema = 'MonthlyI'
# hyphen_index = schema.find("I")
# date1 = '2011-01-01'
# date2 = '2011-01-31'
# st = time.time()
# engine = pg.connect(f"dbname='{index}db' user='postgres' host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com' port='5432' password='swancap123'")
# # ddf = pd.read_sql(f'select * from "{index}"."AllData" where "Date" between \'{date1}\' and \'{date2}\'', con=engine)
# # sql=f"COPY (select *from \"{index}{schema}\".select_datewise(\'{year_start}\',\'{year_end}\')) TO STDOUT WITH DELIMITER ',' CSV HEADER"
# print(f'select * from "{index}{schema}".select_datewise(\'{date1}\',\'{date2}\')')
# ddf = pd.read_sql(f'select * from "{index}{schema}".select_datewise(\'{date1}\',\'{date2}\')',con=engine)
# display(ddf)
# engine.close()
# print(time.time()-st)

In [2]:
# #import the modules
# import psycopg2
# import time
# from io import StringIO
# import pandas as pd
# import os
# from datetime import datetime
# from datetime import date
# import numpy as np
# from tqdm.notebook import tqdm
# import pyspark
# import calendar
# from pyspark.sql import SparkSession
# from pyspark.sql import Row
# from datetime import timedelta
# from pyspark.sql.functions import regexp_replace
# from pyspark.sql.functions import array_contains
# from pyspark.sql.functions import date_format
# import warnings
# warnings.filterwarnings('ignore')

# ## DEFINING FUNCTIONS FOR INDEX UNDERLYING

# def EOD_underlying(index):
#     print("CONVERTING TO EOD")
#     df = pd.read_csv(rf"C:\\Users\\Admin\\Desktop\\{index}_EqData\\{index}" + ".csv")
#     df['Date'] = pd.to_datetime(df['Date'], format='mixed',dayfirst=True)
#     df = df[(df['Time']>='09:15:00') & (df['Time']<='15:30:00')]

#     df = df.sort_values(by=['Date'])

#     final_df = df.copy()
#     final_df = final_df.rename(columns={'Time' : 'Timestamp',
#                                         'Open' : 'Adj_Open',
#                                         'High' : 'Adj_High',
#                                         'Low' : 'Adj_Low',
#                                         'Close' : 'Adj_Close',
#                                         'Volume' : 'Adj_Volume'})
#     final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
#     final_df['Timestamp'] = pd.to_datetime(final_df['Timestamp']).dt.time
#     final_df = final_df.sort_values(by=['Date', 'Timestamp'])
#     final_df['Date'] = final_df['Date'].astype(str)
#     final_df['Timestamp'] = final_df['Timestamp'].astype(str)
#     final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

#     final_df = final_df.set_index("Datetime")

#     ddf = final_df.groupby(['Date', pd.Grouper(freq='B')]).agg({"Adj_Open" : "first", 
#                                                           "Adj_High" : "max",
#                                                           "Adj_Low" : "min",
#                                                           "Adj_Close" : "last", 
#                                                           'Adj_Volume' : 'sum'})
#     ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
#     ddf = ddf.reset_index()
#     ddf['Ticker'] = f"{index}".upper()+'.EQ-NSE'

#     ddf = ddf.sort_values(by=['Datetime'])

#     ddf = ddf.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume'})
#     ddf = ddf[['Ticker','Date','Open','High','Low','Close','Volume']]
#     ddf.to_csv(fr"C:\\Users\\Admin\\Desktop\\{index}_EqData\\{index}_EOD.csv",index=False)
#     os.remove(rf"C:\\Users\\Admin\\Desktop\\{index}_EqData\\{index}" + ".csv")
    
# def fifteen_min_underlying(index):
#     print("CONVERTING TO 5Min")
#     df = pd.read_csv(rf"C:\\Users\\Admin\\Desktop\\{index}_EqData\\{index}" + ".csv")
#     df['Date'] = pd.to_datetime(df['Date'], format='mixed',dayfirst=True)
#     df = df[(df['Time']>='09:15:00') & (df['Time']<='15:30:00')]

#     df = df.sort_values(by=['Date'])

#     final_df = df.copy()
#     final_df = final_df.rename(columns={'Time' : 'Timestamp',
#                                         'Open' : 'Adj_Open',
#                                         'High' : 'Adj_High',
#                                         'Low' : 'Adj_Low',
#                                         'Close' : 'Adj_Close',
#                                         'Volume' : 'Adj_Volume'})
#     final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
#     final_df['Timestamp'] = pd.to_datetime(final_df['Timestamp']).dt.time
#     final_df = final_df.sort_values(by=['Date', 'Timestamp'])
#     final_df['Date'] = final_df['Date'].astype(str)
#     final_df['Timestamp'] = final_df['Timestamp'].astype(str)
#     final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed', dayfirst=True)

#     final_df = final_df.set_index("Datetime")

#     ddf = final_df.groupby(['Date', pd.Grouper(freq='15min')]).agg({"Adj_Open" : "first", 
#                                                           "Adj_High" : "max",
#                                                           "Adj_Low" : "min",
#                                                           "Adj_Close" : "last", 
#                                                           'Adj_Volume' : 'sum'})
#     ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
#     ddf = ddf.reset_index()
#     ddf['Ticker'] = f"{index}".upper()+'.EQ-NSE'

#     ddf = ddf.sort_values(by=['Datetime'])
#     ddf['Time'] = pd.to_datetime(ddf['Datetime']).dt.time

#     ddf = ddf.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume'})
#     ddf = ddf[['Ticker','Date','Time','Open','High','Low','Close','Volume']]
#     ddf.to_csv(fr"C:\\Users\\Admin\\Desktop\\{index}_EqData\\{index}_15min.csv",index=False)
#     os.remove(rf"C:\\Users\\Admin\\Desktop\\{index}_EqData\\{index}" + ".csv")

# def five_min_underlying(index):
#     print("CONVERTING TO 5Min")
#     df = pd.read_csv(rf"C:\\Users\\Admin\\Desktop\\{index}_EqData\\{index}" + ".csv",parse_dates=['Date'])
#     df['Date'] = pd.to_datetime(df['Date'], format='mixed',dayfirst=True)
#     df = df[(df['Time']>='09:15:00') & (df['Time']<='15:30:00')]

#     df = df.sort_values(by=['Date'])

#     final_df = df.copy()
#     final_df = final_df.rename(columns={'Time' : 'Timestamp',
#                                         'Open' : 'Adj_Open',
#                                         'High' : 'Adj_High',
#                                         'Low' : 'Adj_Low',
#                                         'Close' : 'Adj_Close',
#                                         'Volume' : 'Adj_Volume'})
#     final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
#     final_df['Timestamp'] = pd.to_datetime(final_df['Timestamp']).dt.time
#     final_df = final_df.sort_values(by=['Date', 'Timestamp'])
#     final_df['Date'] = final_df['Date'].astype(str)
#     final_df['Timestamp'] = final_df['Timestamp'].astype(str)
#     final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format='mixed',dayfirst=True)

#     final_df = final_df.set_index("Datetime")

#     ddf = final_df.groupby(['Date', pd.Grouper(freq='5min')]).agg({"Adj_Open" : "first", 
#                                                           "Adj_High" : "max",
#                                                           "Adj_Low" : "min",
#                                                           "Adj_Close" : "last", 
#                                                           'Adj_Volume' : 'sum'})
#     ddf.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume']
#     ddf = ddf.reset_index()
#     ddf['Ticker'] = f"{index}".upper()+'.EQ-NSE'

#     ddf = ddf.sort_values(by=['Datetime'])
#     ddf['Time'] = pd.to_datetime(ddf['Datetime']).dt.time

#     ddf = ddf.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume'})
#     ddf = ddf[['Ticker','Date','Time','Open','High','Low','Close','Volume']]
#     ddf.to_csv(fr"C:\\Users\\Admin\\Desktop\\{index}_EqData\\{index}_5min.csv",index=False)
#     os.remove(rf"C:\\Users\\Admin\\Desktop\\{index}_EqData\\{index}" + ".csv")

# def one_min_underlying(index):
#     df = pd.read_csv(rf"C:\\Users\\Admin\\Desktop\\{index}_EqData\\{index}" + ".csv")
#     df['Date'] = pd.to_datetime(df['Date'], format='mixed',dayfirst=True)
#     df = df[(df['Time']>='09:15:00') & (df['Time']<='15:30:00')]
#     df = df.sort_values(by=['Date','Time'])
    
#     df = df[['Ticker','Date','Time','Open','High','Low','Close','Volume']]
#     display(df)
#     df.to_csv(rf"C:\\Users\Admin\Desktop\\{index}_EqData\\{index}.csv",index=False)
    
# ## DEFINING FUNCTIONS FOR INDEX OPTIONS DATA

# def EOD(year1, year2,index,schema,hyphen_index):
#     print("CONVERTING TO EOD")
#     for i in range(int(year1),int(year2)+1):
#         df = pd.read_csv(rf"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv")
#         df = df.rename(columns={'ticker' : 'Ticker',
#                                 'date' : 'Date',
#                                 'time' : 'Time',
#                                 'open' : 'Open',
#                                 'high' : 'High', 
#                                 'low' : 'Low',
#                                 'close' : 'Close',
#                                 'volume' : 'Volume', 
#                                 'Open Int' : 'Open Interest'})
#         df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
#         df = df.sort_values(by=['Date'])

#         ## CHANGE AS PER USER INPUT
#         symbol = index.upper()
#         j='-' + schema[hyphen_index:]
#         schema_find = schema[:hyphen_index].upper()

#         final_df = df.copy()
#         final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
#         final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{index.upper()}'+schema_find, '').str.replace(f'{index.upper()}','').str.replace('CE', '').str.replace('PE', '')
#         final_df['Final_strike'] = final_df['Final_strike'].astype(float)
#         final_df['Option_Type'] = final_df['Ticker'].str[-2:]

#         final_df = final_df.rename(columns={'Time' : 'Timestamp',
#                                             'Open' : 'Adj_Open',
#                                             'High' : 'Adj_High',
#                                             'Low' : 'Adj_Low',
#                                             'Close' : 'Adj_Close',
#                                             'Volume' : 'Adj_Volume',
#                                             'Open Interest' : 'Adj_OI',        
#                                             'Option_type' : 'Option_Type'})
#         final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
#         final_df['Timestamp'] = pd.to_datetime(final_df['Timestamp']).dt.time
#         final_df = final_df.sort_values(by=['Date', 'Timestamp'])
#         final_df['Date'] = final_df['Date'].astype(str)
#         final_df['Timestamp'] = final_df['Timestamp'].astype(str)
#         final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], dayfirst=True)

#         final_df = final_df.set_index("Datetime")
#         final_df['Adj_OI_1'] = final_df['Adj_OI']

#         df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='B')]).agg({"Adj_Open" : "first", 
#                                                               "Adj_High" : "max",
#                                                               "Adj_Low" : "min",
#                                                               "Adj_Close" : "last", 
#                                                               'Adj_Volume' : 'sum',
#                                                               'Adj_OI' : 'first',
#                                                               'Adj_OI_1' : 'last'})
#         df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
#         df_eod = df_eod.reset_index()
#         df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
#         df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol + schema_find + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
#         df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol + schema_find + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

#         df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
#         df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
#         ddf = df_eod.copy()
#         ddf['Date'] = pd.to_datetime(ddf['Date'],dayfirst=True)
#         ## CHECKING IF NULL VALUES
#         ddf_OI = ddf.copy()
#         ddf_OI['New_OI'] = ddf_OI['Last_OI']
#         ddf_OI = ddf_OI.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
#         ddf_OI = ddf_OI.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
#         ddf_OI = ddf_OI[['Ticker','Date','Open','High','Low','Close','Volume','Open_Interest']]
#         ddf_OI.to_csv(fr"C:\users\admin\desktop\\{index}_{schema}_Data\\{index}_{schema}.csv",mode='a',header= not os.path.exists(fr"C:\users\admin\desktop\\{index}_{schema}_Data\\{index}_{schema}.csv"),index=False)
#         os.remove(rf"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv")

# def fifteen_min(year1, year2,index,schema,hyphen_index):
#     print("CONVERTING TO 15Min")
#     for i in range(int(year1),int(year2)+1):
#         df = pd.read_csv(rf"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv")
#         df = df.rename(columns={'ticker' : 'Ticker',
#                                 'date' : 'Date',
#                                 'time' : 'Time',
#                                 'open' : 'Open',
#                                 'high' : 'High', 
#                                 'low' : 'Low',
#                                 'close' : 'Close',
#                                 'volume' : 'Volume', 
#                                 'Open Int' : 'Open Interest'})
#         df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
#         df = df.sort_values(by=['Date'])

#         ## CHANGE AS PER USER INPUT
#         symbol = index.upper()
#         j='-' + schema[hyphen_index:]
#         schema_find = schema[:hyphen_index].upper()

#         final_df = df.copy()
#         final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
#         final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{index.upper()}'+schema_find, '').str.replace(f'{index.upper()}','').str.replace('CE', '').str.replace('PE', '')
#         final_df['Final_strike'] = final_df['Final_strike'].astype(float)
#         final_df['Option_Type'] = final_df['Ticker'].str[-2:]

#         final_df = final_df.rename(columns={'Time' : 'Timestamp',
#                                             'Open' : 'Adj_Open',
#                                             'High' : 'Adj_High',
#                                             'Low' : 'Adj_Low',
#                                             'Close' : 'Adj_Close',
#                                             'Volume' : 'Adj_Volume',
#                                             'Open Interest' : 'Adj_OI',        
#                                             'Option_type' : 'Option_Type'})
#         final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
#         final_df['Timestamp'] = pd.to_datetime(final_df['Timestamp']).dt.time
#         final_df = final_df.sort_values(by=['Date', 'Timestamp'])
#         final_df['Date'] = final_df['Date'].astype(str)
#         final_df['Timestamp'] = final_df['Timestamp'].astype(str)
#         final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], dayfirst=True)

#         final_df = final_df.set_index("Datetime")
#         final_df['Adj_OI_1'] = final_df['Adj_OI']

#         df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='15min')]).agg({"Adj_Open" : "first", 
#                                                               "Adj_High" : "max",
#                                                               "Adj_Low" : "min",
#                                                               "Adj_Close" : "last", 
#                                                               'Adj_Volume' : 'sum',
#                                                               'Adj_OI' : 'first',
#                                                               'Adj_OI_1' : 'last'})
#         df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
#         df_eod = df_eod.reset_index()
#         df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
#         df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol + schema_find + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
#         df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol + schema_find + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

#         df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
#         df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
#         ddf = df_eod.copy()
#         ddf['Time'] = pd.to_datetime(ddf['Date']).dt.time
#         ddf['Date'] = pd.to_datetime(ddf['Date'],dayfirst=True).dt.date
#         ## CHECKING IF NULL VALUES
#         ddf_OI = ddf.copy()
#         ddf_OI['New_OI'] = ddf_OI['Last_OI']
#         ddf_OI = ddf_OI.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
#         ddf_OI = ddf_OI.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
#         ddf_OI = ddf_OI[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
#         ddf_OI.to_csv(fr"C:\users\admin\desktop\\{index}_{schema}_Data\\{index}_{schema}.csv",mode='a',header= not os.path.exists(fr"C:\users\admin\desktop\\{index}_{schema}_Data\\{index}_{schema}.csv"),index=False)
#         os.remove(rf"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv")

# def five_min(year1,year2,index,schema,hyphen_index):
#     print("CONVERTING TO 5Min")
#     for i in range(int(year1),int(year2)+1):
#         df = pd.read_csv(rf"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv")
#         df = df.rename(columns={'ticker' : 'Ticker',
#                                 'date' : 'Date',
#                                 'time' : 'Time',
#                                 'open' : 'Open',
#                                 'high' : 'High', 
#                                 'low' : 'Low',
#                                 'close' : 'Close',
#                                 'volume' : 'Volume', 
#                                 'Open Int' : 'Open Interest'})
#         df['Date'] = pd.to_datetime(df['Date'], format = 'mixed', dayfirst=True)
#         df = df.sort_values(by=['Date'])

#         ## CHANGE AS PER USER INPUT
#         symbol = index.upper()
#         j='-' + schema[hyphen_index:]
#         schema_find = schema[:hyphen_index].upper()

#         final_df = df.copy()
#         final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
#         final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{index.upper()}'+schema_find, '').str.replace(f'{index.upper()}','').str.replace('CE', '').str.replace('PE', '')
#         final_df['Final_strike'] = final_df['Final_strike'].astype(float)
#         final_df['Option_Type'] = final_df['Ticker'].str[-2:]

#         final_df = final_df.rename(columns={'Time' : 'Timestamp',
#                                             'Open' : 'Adj_Open',
#                                             'High' : 'Adj_High',
#                                             'Low' : 'Adj_Low',
#                                             'Close' : 'Adj_Close',
#                                             'Volume' : 'Adj_Volume',
#                                             'Open Interest' : 'Adj_OI',        
#                                             'Option_type' : 'Option_Type'})
#         final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
#         final_df['Timestamp'] = pd.to_datetime(final_df['Timestamp']).dt.time
#         final_df = final_df.sort_values(by=['Date', 'Timestamp'])
#         final_df['Date'] = final_df['Date'].astype(str)
#         final_df['Timestamp'] = final_df['Timestamp'].astype(str)
#         final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], format = 'mixed',dayfirst=True)

#         final_df = final_df.set_index("Datetime")
#         final_df['Adj_OI_1'] = final_df['Adj_OI']

#         df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='5min')]).agg({"Adj_Open" : "first", 
#                                                               "Adj_High" : "max",
#                                                               "Adj_Low" : "min",
#                                                               "Adj_Close" : "last", 
#                                                               'Adj_Volume' : 'sum',
#                                                               'Adj_OI' : 'first',
#                                                               'Adj_OI_1' : 'last'})
#         df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
#         df_eod = df_eod.reset_index()
#         df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
#         df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol + schema_find + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
#         df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol + schema_find + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

#         df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
#         df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
#         ddf = df_eod.copy()
#         ddf['Time'] = pd.to_datetime(ddf['Date']).dt.time
#         ddf['Date'] = pd.to_datetime(ddf['Date'],format = 'mixed',dayfirst=True).dt.date
#         ## CHECKING IF NULL VALUES
#         ddf_OI = ddf.copy()
#         ddf_OI['New_OI'] = ddf_OI['Last_OI']
#         ddf_OI = ddf_OI.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
#         ddf_OI = ddf_OI.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
#         ddf_OI = ddf_OI[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
#         ddf_OI.to_csv(fr"C:\users\admin\desktop\\{index}_{schema}_Data\\{index}_{schema}.csv",mode='a',header= not os.path.exists(fr"C:\users\admin\desktop\\{index}_{schema}_Data\\{index}_{schema}.csv"),index=False)
#         os.remove(rf"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv")

# def one_min(year1,year2,index,schema,hyphen_index): 
#     for i in range(int(year1),int(year2)+1):
#         df = pd.read_csv(rf"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv")

#         ddf['Date'] = pd.to_datetime(ddf['Date'], dayfirst=True)
#         ddf = ddf.sort_values(by=['Date'])
#         ddf = ddf[(ddf['Time']>=time1) & ((ddf['Time']<=time2))]
#         ddf = ddf.sort_values(by=['Date','Time'])
#         ddf.reset_index(drop=True,inplace=True)
#         ddf = ddf[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
#         ddf.to_csv(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_1min.csv", mode='a', header = not os.path.exists(fr"C:\\users\\{admin_path}\\desktop\\Stocks_Data\\{symbol.upper()}_Opt_1min.csv"), index=False)

#         df = df.rename(columns={'ticker' : 'Ticker',
#                                 'date' : 'Date',
#                                 'time' : 'Time',
#                                 'open' : 'Open',
#                                 'high' : 'High', 
#                                 'low' : 'Low',
#                                 'close' : 'Close',
#                                 'volume' : 'Volume', 
#                                 'Open Int' : 'Open Interest'})
#         df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
#         df = df.sort_values(by=['Date'])

#         ## CHANGE AS PER USER INPUT
#         symbol = index.upper()
#         j='-' + schema[hyphen_index:]
#         schema_find = schema[:hyphen_index].upper()

#         final_df = df.copy()
#         final_df['Final_strike'] = final_df['Ticker'].str.replace(j, '')
#         final_df['Final_strike'] = final_df['Final_strike'].str.replace(f'{index.upper()}'+schema_find, '').str.replace(f'{index.upper()}','').str.replace('CE', '').str.replace('PE', '')
#         final_df['Final_strike'] = final_df['Final_strike'].astype(float)
#         final_df['Option_Type'] = final_df['Ticker'].str[-2:]

#         final_df = final_df.rename(columns={'Time' : 'Timestamp',
#                                             'Open' : 'Adj_Open',
#                                             'High' : 'Adj_High',
#                                             'Low' : 'Adj_Low',
#                                             'Close' : 'Adj_Close',
#                                             'Volume' : 'Adj_Volume',
#                                             'Open Interest' : 'Adj_OI',        
#                                             'Option_type' : 'Option_Type'})
#         final_df['Date'] = pd.to_datetime(final_df['Date'],dayfirst=True).dt.date
#         final_df['Timestamp'] = pd.to_datetime(final_df['Timestamp']).dt.time
#         final_df = final_df.sort_values(by=['Date', 'Timestamp'])
#         final_df['Date'] = final_df['Date'].astype(str)
#         final_df['Timestamp'] = final_df['Timestamp'].astype(str)
#         final_df['Datetime'] = pd.to_datetime(final_df['Date'] + ' ' + final_df['Timestamp'], dayfirst=True)

#         final_df = final_df.set_index("Datetime")
#         final_df['Adj_OI_1'] = final_df['Adj_OI']

#         df_eod = final_df.groupby(['Final_strike', 'Option_Type', pd.Grouper(freq='1min')]).agg({"Adj_Open" : "first", 
#                                                               "Adj_High" : "max",
#                                                               "Adj_Low" : "min",
#                                                               "Adj_Close" : "last", 
#                                                               'Adj_Volume' : 'sum',
#                                                               'Adj_OI' : 'first',
#                                                               'Adj_OI_1' : 'last'})
#         df_eod.columns = ["Adj_Open", "Adj_High", "Adj_Low", "Adj_Close", 'Adj_Volume', 'First_OI', 'Last_OI']
#         df_eod = df_eod.reset_index()
#         df_eod['rem'] = df_eod['Final_strike']%df_eod['Final_strike'].astype(int)
#         df_eod.loc[df_eod['rem'] == 0, 'Ticker'] = symbol + schema_find + j +  df_eod['Final_strike'].astype(int).astype(str) + df_eod['Option_Type']
#         df_eod.loc[df_eod['rem'] != 0, 'Ticker'] = symbol + schema_find + j + df_eod['Final_strike'].round(2).astype(str) + df_eod['Option_Type'] 

#         df_eod = df_eod.sort_values(by=['Datetime', 'Final_strike'])
#         df_eod = df_eod.rename(columns={'Datetime' : 'Date'})
#         ddf = df_eod.copy()
#         ddf['Time'] = pd.to_datetime(ddf['Date']).dt.time
#         ddf['Date'] = pd.to_datetime(ddf['Date'],dayfirst=True).dt.date
#         ## CHECKING IF NULL VALUES
#         ddf_OI = ddf.copy()
#         ddf_OI['New_OI'] = ddf_OI['Last_OI']
#         ddf_OI = ddf_OI.rename(columns={'Adj_Open':'Open','Adj_High':'High','Adj_Low':'Low','Adj_Close':'Close','Adj_Volume':'Volume','New_OI':'Open_Interest'})
#         ddf_OI = ddf_OI.drop(['First_OI','Last_OI','Option_Type','Final_strike','rem'],axis=1)
#         ddf_OI = ddf_OI[['Ticker','Date','Time','Open','High','Low','Close','Volume','Open_Interest']]
#         ddf_OI.to_csv(fr"C:\users\admin\desktop\\{index}_{schema}_Data\\{index}_{schema}.csv",mode='a',header= not os.path.exists(fr"C:\users\admin\desktop\\{index}_{schema}_Data\\{index}_{schema}.csv"),index=False)
#         os.remove(rf"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv")

# def option_data_inputs(index,date1,date2,conversion):    
#     schema = input("ENTER THE SCHEMA OF WHOSE DATA YOU WANT - \nMonthlyI , MonthlyII , WeeklyI , QuarterlyI and so on\n")
#     hyphen_index = schema.find("I")
#     conn = psycopg2.connect(database=f"{index}db",
#                             user='postgres', password='swancap123',
#                             host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
#                             )
#     conn.autocommit = True
#     cursor = conn.cursor()
#     buffer = StringIO()
#     st=time.time()
#     ## CREATING A DIRECTORY OF THE REQUIRED INDEX AND SCHEMA
#     if not os.path.exists(rf"C:\\users\\admin\\desktop\\{index}_{schema}_Data\\"):
#         os.makedirs(rf"C:\users\admin\desktop\\{index}_{schema}_Data\\")

#     date1 = datetime.strptime(date1, "%Y-%m-%d").date()
#     date2 = datetime.strptime(date2, "%Y-%m-%d").date()
#     year1 = date1.year
#     year2 = date2.year
#     print("\nGENERATING OPTIONS DATA")
#     for i in range(int(year1),int(year2)+1):
#         ddate1 = '-01-01'
#         ddate2 = '-12-31'
#         year_start = str(str(i)+ddate1)
#         year_end = str(str(i)+ddate2)
#         year_start = datetime.strptime(year_start, "%Y-%m-%d").date()
#         year_end = datetime.strptime(year_end, "%Y-%m-%d").date()
#         if date1 > year_start :
#             year_start = date1
#         else:
#             year_start = year_start
#         if year_end > date2 :
#             year_end = date2
#         else:
#             year_end = year_end
#         sql=f"COPY (select *from \"{index}{schema}\".select_datewise(\'{year_start}\',\'{year_end}\')) TO STDOUT WITH DELIMITER ',' CSV HEADER"
#         print("Generating data from ", year_start , " to " , year_end)
#         with open(f"C:\\Users\\Admin\\Desktop\\{index}_{schema}_Data\\{index}_{schema}_"+str(i)+".csv", "w") as file:
#             cursor.copy_expert(sql, file)
#     if conversion == 'E':
#         df = EOD(year1, year2,index,schema,hyphen_index)

#     elif conversion == '15':
#         df = fifteen_min(year1,year2,index,schema,hyphen_index)

#     elif conversion == '5':
#         df = five_min(year1,year2,index,schema,hyphen_index)

#     elif conversion == '1':
#         df = one_min(year1,year2,index,schema,hyphen_index)

#     conn.commit()
#     conn.close()
#     et=time.time()

#     elapsed_time=et-st;
#     print("OPTIONS DATA GENERATED!")
#     print("elapsed_time:",elapsed_time)

# def underlying_inputs(index,date1,date2,conversion):
#     start_date = datetime.strptime(date1,"%Y-%m-%d")
#     end_date = datetime.strptime(date2,"%Y-%m-%d")
#     start_date=start_date.date()
#     end_date = end_date.date()
#     print("\nGENERATING UNDERLYING DATA")
#     st = time.time()
#     conn = psycopg2.connect(database="IndexEQ",
#                             user='postgres', password='swancap123',
#                             host='swandatabase.cfehmk2wtejq.ap-south-1.rds.amazonaws.com', port='5432'
#     )

#     conn.autocommit = True
#     cursor = conn.cursor()
#     buffer = StringIO()

#     sql = f"COPY (SELECT * from \"{index}\".\"AllData\" where \"Date\" BETWEEN \'{start_date}' AND \'{end_date}\') TO STDOUT WITH DELIMITER ',' CSV HEADER" 

#     if not os.path.exists(rf"C:\\users\\admin\\desktop\\{index}_EqData\\"):
#         os.makedirs(rf"C:\users\admin\desktop\\{index}_EqData\\")        

#     with open(fr"C:\users\admin\desktop\\{index}_EqData\\{index}" + ".csv","w") as file:
#         cursor.copy_expert(sql,file)

#     if conversion == '1':
#         pass
#         # df = one_min_underlying(index)

#     elif conversion == '5':
#         df = five_min_underlying(index)

#     elif conversion == '15':
#         df = fifteen_min_underlying(index)

#     elif conversion == 'E':
#         df = EOD_underlying(index)

#     conn.commit()
#     conn.close()
#     et=time.time()

#     elapsed_time=et-st;
#     print("UNDERLYING DATA GENERATED!")
#     print("elapsed_time:",elapsed_time)


# data = input("Enter O for Options data, U for Underlying data, B for Both the data, E for Exiting ")

# if data == 'o' or data == 'O' or data == 'u' or data == 'U' or data == 'b' or data == 'B':    
#     index = input("Enter the index you want in the format below - \nBankNifty\nNifty\nFinNifty\n")
#     date1 = input("Enter start date as YYYY-MM-DD ")
#     date2 = input("Enter end date as YYYY-MM-DD ")
#     conversion = input("Enter 1 for 1 minute, 5 for 5 minutes, 15 for 15 minutes, E for EOD\n")

# elif data == 'e' or data == 'E':
#     print("Exit!")

# else:
#     print("Wrong option")
    
# if data == 'O' or data == 'o':
#     option_data_inputs(index,date1,date2,conversion)

# elif data == 'U' or data == 'u':
#     underlying_inputs(index,date1,date2,conversion)

# elif data == 'B' or data == 'b':
#     option_data_inputs(index,date1,date2,conversion)
#     underlying_inputs(index,date1,date2,conversion)

# print("\nCOMPLETED.")
